<a href="https://colab.research.google.com/github/OculusMode/ILL/blob/main/experimental/random_subspaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from numpy import random, linalg as LA
import matplotlib.pyplot as plt

In [6]:
TOTAL_VECTORS = 64
VECTOR_SIZE = 8
# random.seed(45)
random_vectors = random.rand(TOTAL_VECTORS, VECTOR_SIZE)

In [13]:
def get_random_decomposition(random_vectors):
  # finding 8 random indices from 64 to choose random vectors
  # random.seed(42)
  random_indices = random.choice(TOTAL_VECTORS, size = VECTOR_SIZE)
  while True:
    random_chosen_vectors = random_vectors[random_indices]
    random_basis, _ = LA.qr(random_chosen_vectors)
    # making sure we got 8 basis
    if LA.matrix_rank(random_basis) == VECTOR_SIZE:
      break
  # number of subspaces to choose
  # random.seed(15)
  total_splits = random.randint(2, VECTOR_SIZE)
  # split points to split array
  # random.seed(10)
  split_points = np.random.choice(VECTOR_SIZE - 2, total_splits - 1, replace = False) + 1
  split_points.sort()
  return np.split(random_basis, split_points)


def projected_vector(basis, vector):
  # our basis vector here is row vector so (1, 10) => we want something like (10,1)
  return np.matmul(basis.T, vector)

def projection(basis, vector):
  return LA.norm(projected_vector(basis, vector))

def lifting(initial_vector, basis_matrix, energy):
# projection of vector on subspace
  p1 = np.matmul(np.matmul(basis_matrix.T, basis_matrix), initial_vector)
  p2 = initial_vector - p1
  py = energy * p1 / (LA.norm(p1) + 0.001) + (1 - energy**2)**0.5 * p2 / (LA.norm(p2) + 0.001)
  py = py/LA.norm(py)
  return py

In [15]:
def run_test(epoch=10, no_decompositions=50):
  decompositions = []
  for i in range(no_decompositions):
    decompositions.append(get_random_decomposition(random_vectors))
  signal = random.rand(VECTOR_SIZE)
  signal = signal/LA.norm(signal)
  energies = []
  for decomposition in decompositions:
    e = []
    for subspace in decomposition:
      e.append(projection(subspace.T, signal))
    energies.append(e)
  random_vector = random.rand(VECTOR_SIZE)
  random_vector = random_vector/LA.norm(random_vector)
  for i in range(epoch):
    x = np.arange(no_decompositions)
    for idx_decomp, d in enumerate(decompositions):
      for idx_ss, subspace in enumerate(d):
        # print(subspace.shape, signal.shape)
        random_vector = lifting(random_vector, subspace, energies[idx_decomp][idx_ss])
  # print(LA.norm(random_vector - signal))
  if LA.norm(random_vector - signal) < 0.05:
    return 1
  return 0


In [20]:
decomps = range(10, 60, 10)

tot = 1000
for d_no in decomps:
  tot_p = []
  for _ in range(5):
    # to have fixed output
    # random.seed(42)
    tot_succ=0
    for i in range(tot):
      tot_succ+=run_test(no_decompositions=d_no)
    # print(tot, tot_succ/tot)
    tot_p.append(tot_succ/tot)
  print(tot_p)
# 28 min

"""
DO IT FOR 100/1000 times
AND FOR INCREMENTING NUMBER OF decompositions
"""


[0.978, 0.978, 0.987, 0.983, 0.992]
[0.985, 0.992, 0.987, 0.985, 0.982]
[0.989, 0.986, 0.99, 0.991, 0.982]
[0.991, 0.984, 0.987, 0.982, 0.987]
[0.983, 0.989, 0.985, 0.983, 0.992]


'\nDO IT FOR 100/1000 times\nAND FOR INCREMENTING NUMBER OF decompositions\n'

In [21]:
res = [
  [0.978, 0.978, 0.987, 0.983, 0.992], # 
  [0.985, 0.992, 0.987, 0.985, 0.982], # 
  [0.989, 0.986, 0.99, 0.991, 0.982], # 
  [0.991, 0.984, 0.987, 0.982, 0.987], # 
  [0.983, 0.989, 0.985, 0.983, 0.992], # 
]
[sum(i)/5 for i in res]

[0.9836, 0.9862, 0.9875999999999999, 0.9862, 0.9864]

In [ ]:
# totally random
def get_random_energy(vector_size, random_signal):
  subspace_size = random.randint(1, vector_size + 1)
  while True:
    a = random.rand(vector_size, subspace_size)
    basis, _ = LA.qr(a)
    if LA.matrix_rank(basis) == subspace_size:
      break
  # print(projection(q, random_signal))
  return projection(basis, random_signal)
  
# so we will create hella data. honestly, basically 50 energies for each signal.
def get_data(vector_size, no_of_rows, no_of_columns):
  
  # starting with creating bunch of subspaces (no_of_columns)
  random_vectors = random.rand(256, vector_size)
  subspaces_basis = []

  # adding N number of subspaces in a list(then we can count energy for random signals)
  for row in range(no_of_columns):
    subspace_size = random.randint(1, vector_size + 1)
    random_indices = random.choice(256, size = subspace_size)
    while True:
      a = random_vectors[random_indices].T
      basis, _ = LA.qr(a)
      if LA.matrix_rank(basis) == subspace_size:
        break
    subspaces_basis.append(basis)
  
  Y = []
  X = []
  for _ in range(no_of_rows):
    random_signal = random.rand(vector_size)
    random_signal = random_signal/np.sum(random_signal)
    Y.append(random_signal)
    X.append([projection(basis, random_signal) for basis in subspaces_basis])
  X = np.array(X)
  return X, np.array(Y)
X, Y = get_data(8, 10_000, 50)

In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics
import tensorflow as tf

X_train = X[:7000]
X_test = X[7000:]
y_train = Y[:7000]
y_test = Y[7000:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
model = Sequential()
model.add(Dense(50, input_dim=50, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(8, activation='sigmoid'))
# compile the keras model
# def my_metric_fn(y_true, y_pred):
#   print('=>',y_true.shape, y_pred.shape)
#   return tf.norm(y_true - y_pred)

model.compile(
    optimizer='adam', 
    loss='mean_squared_error',
    metrics=[
        'accuracy',
        'mean_squared_error'
    ])
# fit the keras model on the dataset
# , verbose=0
history = model.fit(X_train, y_train, epochs=150, batch_size=8)
# evaluate the keras model
_, a, b = model.evaluate(X_test, y_test)
# print('Accuracy: %.2f' % (mse*100))

(7000, 50) (3000, 50) (7000, 8) (3000, 8)
Epoch 1/150
875/875 [==============================] - 2s 2ms/step - loss: 0.0077 - accuracy: 0.1907 - mean_squared_error: 0.0077
Epoch 2/150
875/875 [==============================] - 2s 2ms/step - loss: 0.0045 - accuracy: 0.3330 - mean_squared_error: 0.0045
Epoch 3/150
875/875 [==============================] - 2s 2ms/step - loss: 0.0036 - accuracy: 0.3869 - mean_squared_error: 0.0036
Epoch 4/150
875/875 [==============================] - 2s 2ms/step - loss: 0.0029 - accuracy: 0.4529 - mean_squared_error: 0.0029
Epoch 5/150
875/875 [==============================] - 2s 2ms/step - loss: 0.0023 - accuracy: 0.5169 - mean_squared_error: 0.0023
Epoch 6/150
875/875 [==============================] - 2s 2ms/step - loss: 0.0018 - accuracy: 0.5614 - mean_squared_error: 0.0018
Epoch 7/150
875/875 [==============================] - 2s 2ms/step - loss: 0.0014 - accuracy: 0.6183 - mean_squared_error: 0.0014
Epoch 8/150
875/875 [===========================

In [ ]:
avg = 0
for i in range(7001, 10_000):
  avg += np.sum(np.abs(model.predict(X[i].reshape(1, 50)) - Y[i]))
# print('Accuracy: %.2f' % (mse*100))
print(avg/3000)

0.059595319544573054


In [ ]:
model.predict(X[8000].reshape(1, 50)), Y[8000]

(array([[0.06986117, 0.14935306, 0.08382797, 0.23959151, 0.04598585,
         0.06464773, 0.13845652, 0.20590544]], dtype=float32),
 array([0.0667871 , 0.15769052, 0.08787615, 0.23726378, 0.03377103,
        0.06671555, 0.13498634, 0.21490952]))

In [ ]:
history